In [1]:
import pandas as pd
import yfinance as yf

In [3]:
initial_companies = ['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY', 'INTU', 'V', 'MA', 'BX', 'TTE', 'GOOGL']

In [4]:
yf_objs = []
for company in initial_companies:
    yf_objs.append(yf.Ticker(company))

In [6]:
msft.history(interval='5m')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-10-09 09:30:00-04:00,324.750000,325.199799,323.589996,324.149994,1512636,0.0,0.0
2023-10-09 09:35:00-04:00,324.174988,324.540009,323.179993,324.174988,590075,0.0,0.0
2023-10-09 09:40:00-04:00,324.183807,325.714996,323.850006,325.570007,435328,0.0,0.0
2023-10-09 09:45:00-04:00,325.579987,326.859985,325.470001,326.839996,453587,0.0,0.0
2023-10-09 09:50:00-04:00,326.829987,327.179993,326.190002,326.850006,327676,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-07 15:35:00-05:00,361.239990,361.730011,361.230011,361.720001,236374,0.0,0.0
2023-11-07 15:40:00-05:00,361.709991,361.829987,361.579987,361.769989,297390,0.0,0.0
2023-11-07 15:45:00-05:00,361.789886,361.890015,361.320007,361.369995,333845,0.0,0.0


In [25]:
for comp_obj in yf_objs:
    print(comp_obj.history(interval='5m').shape)

(1716, 7)
(1716, 7)
(1716, 7)
(1716, 7)
(1709, 7)
(1716, 7)
(1716, 7)
(1716, 7)
(1716, 7)
(1716, 7)
(1716, 7)


In [26]:
yf_objs[0].history(interval='5m')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-10-09 09:30:00-04:00,587.650024,589.309998,583.869995,583.955017,56251,0.0,0.0
2023-10-09 09:35:00-04:00,583.210022,584.614990,582.409973,584.380005,26848,0.0,0.0
2023-10-09 09:40:00-04:00,584.429993,586.650024,583.090027,586.289978,20933,0.0,0.0
2023-10-09 09:45:00-04:00,586.750000,587.619995,585.140015,587.549988,21245,0.0,0.0
2023-10-09 09:50:00-04:00,587.609985,587.830017,585.669983,587.390015,15559,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-07 15:35:00-05:00,637.919983,638.049988,637.630005,637.820984,6104,0.0,0.0
2023-11-07 15:40:00-05:00,637.909973,638.080017,637.775024,637.940002,5276,0.0,0.0
2023-11-07 15:45:00-05:00,637.674988,637.820007,636.859985,637.289978,8986,0.0,0.0
